In [ ]:
#clean files and load into dynamo db

# process the data lines only and keep those which pass the grep with nothing
#grep -e 'CTGTCTCTTATACACATCT' -e 'AAAGAGTGT' -e 'TATGCAGT' -e 'GCTCATGA' -e 'TCATGAGC' -e 'ACACGTCTGAA' -e 'CGTCGTGTAGGG' -e 'GATCGGAAGAGC'

In [ ]:
import gzip,time

class Adapter_Clean:
    def __init__(self):
        self.num_read=0
        self.N_pos_dict={}
        self.nuc_dict={}
        self.low_phred={}
        self.adapters = {'CTGTCTCTTATACACATCT':0, 'GATCGGAAGAGC':0, 'CTCCAGTCAC':0, 'CGTCGTGTAGGG':0, 'ACACGTCTGAA':0, 'AAAGAGTGT':0,
           'TCATGAGC':0, 'GCTCATGA':0, 'TATGCAGT':0}
        self.PATH="/home/dc/Downloads/"
        self.first_file="empty_S316_L008_R1_001.fastq.gz"
        self.second_file = "empty_S316_L008_R2_001.fastq.gz"
        self.testheader = b'@K00153:508:HN5HMBBXX:8:1101:1641:1138 1:N:0:AAGANGCA+TATGCAGT@K00153:508:HN5HMBBXX:8:1101:1641:1138 1:N:0:AAGANGCA+TATGCAGT'
        self.decoded_testheader = self.testheader.decode('UTF-8')
        self.output_file = self.PATH+"single_output.fastq.gz"
        self.output_filehandle = gzip.open(self.output_file,'wb+')
        self.save=True
        
    def process_header(self,line,num_at,adapters):
        """
        process the header and add any multiplexing headers we see to the adpaters dictionary
        input: assumes concatenated header with 2 @ signs like:
        
        """
        #print("process_header line:",line)
        header = line.split("@")
        #to debug use the test string and python console
        adapters_first = header[1].split()[-1].split(':')[-1].split('+')
        adapters_second = header[2].split()[-1].split(':')[-1].split('+')
    
        for x in adapters_first:
            if x not in self.adapters:
                print("adding multiplixing adapter:",x)
                adapters[x]=0
        for x in adapters_second:
            if x not in self.adapters:
                print("adding multiplixing adapter:",x)
                adapters[x]=0
                
    def test_process_header(self):
        '''
        TBD
        '''
        process_header()
        
    def process_plus(self,read_line,num_plus):
        """
        does nothing but count plus signs for QC
        """
        #print('process_plus')
        if (read_line.strip()=="++"):
            num_plus+=1
        else:
            print("error in plus sign delimiter")


    def process_DNA(self,line,adapters):
        """
        1)find lines with N; add those to dictionary of pos where N is. Use this to generate graph of 
        where N is. 
        2)counts of Nucleotides, GC count. 
        """
        #print("process_DNA:")
        #get adapter stats
        for x in adapters:
            #print('processing adapter:',x)
            if x in line:
                #print('process_DNA setting self.save false found adapter!',x)
                self.save=False
                adapters[x] += 1
            
        for i,j in enumerate(list(line)):
            if(j=='N'):
                self.save=False
                #print("process_DNA N setting self.save false:",i)
                if not i in self.N_pos_dict:
                    self.N_pos_dict[i]=1
                else:
                    self.N_pos_dict[i] +=1
        #stats for nucleotides
            if not j in self.nuc_dict:
                self.nuc_dict[j] = 1
            else:
                self.nuc_dict[j]+=1
        
        
    def process_phred(self,line):
        """
        find lines which have phred<20 and store those line positions in low_phred
        return save=false if phred>20
        """
        phred_good=True
        plot_quality=[ord(x)-33 for x in line]
        for i,j in enumerate (plot_quality):
            if j<20:
                self.save=False
                phred_good=False
                if not i in self.low_phred:
                    self.low_phred[i]=1
                else:
                    self.low_phred[i] +=1
        #if phred_good==False:
        #    print("phred bad < 20")
        #else:
        #    print('phred good')
        #this plots per line you can accumulate these to produce a heat map
        #print(plot_quality)
        #plt.plot(plot_quality)
        #plt.show()
        #plt.gcf().clear()
        
    def plot_phred(self):
        '''
        plot the phred dicttionary for all data
        not sure how this works? generate png? 
        '''
        
    def merge_single_file(self):
        '''
        create single gz file from first_file,second_file
        '''
        startTime = time.time()
    
        with gzip.open(PATH+first_file,"rb") as fh1, gzip.open(PATH+second_file,"rb") as fh2,gzip.open(single_file, 'wb') as f_out:
            for x,y in zip(fh1,fh2):
                readline1 = x.strip()
                readline2 = x.strip()
                full_read = readline1 + readline2
                #print(full_read)
                f_out.write(full_read)
        f_out.close()
        fh1.close()
        fh2.close()
        print(time.time()-startTime)    
        
    def save_file(self,list_strings):
        '''
        write save buffer to file
        '''
        #print('saving buffer!!!!!')
        for x in list_strings:
            self.output_filehandle.write(x)
            self.output_filehandle.write('\n'.encode("utf-8"))
        
    def process_files_for_stats(self):
        '''
        read all lines and make stats for adapters
        '''
        
        read_line = ""
        num_at = 0
        num_plus = 0
        save_buffer = []
        
        function_lookup = {0:(self.process_header,read_line,num_at,self.adapters),1:(self.process_DNA,read_line,self.adapters),2:(self.process_plus,read_line,num_plus),3:(self.process_phred,read_line)}
        with gzip.open(self.PATH+self.first_file,"rb") as fh1, gzip.open(self.PATH+self.second_file,"rb") as fh2:
            for x,y in zip(fh1,fh2):
                readline1 = x.strip() ; readline2 = y.strip()
                read_line = (readline1 + readline2).decode('UTF-8')
                save_buffer.append(readline1+readline2)
                func = function_lookup.get(num_at%4)
                #print("read_line:",read_line);print("num_at:",num_at, num_at%4, self.save)
                if(num_at % 4 ==0):
                    func[0](read_line,num_at,self.adapters)
                elif(num_at%4==1):
                    func[0](read_line,self.adapters)
                elif((num_at%4)==2):
                    func[0](read_line,num_plus)
                else:
                    #process phred
                    func[0](read_line)
                    if (self.save==True and num_at>0):
                        self.save_file(save_buffer)
                    save_buffer = []
                    self.save=True
                num_at += 1
        self.output_filehandle.close()
    
    def test_process_files_for_stats(self):
        self.first_file = 'test1.fastq.gz'
        self.second_file = 'test2.fastq.gz'
        a.process_files_for_stats()
        
if __name__=="__main__":
    a = Adapter_Clean()
    a.process_files_for_stats()
    #a.test_process_files_for_stats()
    

adding multiplixing adapter: AAGANGCA
adding multiplixing adapter: TATGCCGT
adding multiplixing adapter: AAGAGGCA
adding multiplixing adapter: TATGCAGG
adding multiplixing adapter: TATGCAGC
adding multiplixing adapter: AAGGGGCA
adding multiplixing adapter: AAGCGGCA
adding multiplixing adapter: TATGCGGT
adding multiplixing adapter: TCTGCAGT
adding multiplixing adapter: TATGAAGT
adding multiplixing adapter: AATGCAGT
adding multiplixing adapter: AAGAGGCC
adding multiplixing adapter: TATGCAGA
adding multiplixing adapter: TATGNAGT
adding multiplixing adapter: AAGAGACA
adding multiplixing adapter: AAAAGGCA
adding multiplixing adapter: TATGCTGT
adding multiplixing adapter: CAGAGGCA
adding multiplixing adapter: AAGACGCA
adding multiplixing adapter: TGTGCAGT
adding multiplixing adapter: TATCCAGT
adding multiplixing adapter: TACGCAGT
adding multiplixing adapter: ACGAGGCA
adding multiplixing adapter: GAGAGGCA
adding multiplixing adapter: AAGAAGCA
adding multiplixing adapter: AGGAGGCA
adding multi

In [ ]:
print(a..__dict__.keys())

In [ ]:
print (Adapter_Clean.adapter)

In [ ]:
import gzip,time
adapters = {'CTGTCTCTTATACACATCT':0, 'GATCGGAAGAGC':0, 'CTCCAGTCAC':0, 'CGTCGTGTAGGG':0, 'ACACGTCTGAA':0, 'AAAGAGTGT':0,
           'TCATGAGC':0, 'GCTCATGA':0, 'TATGCAGT':0}

PATH="/home/dc/Downloads/"
first_file="empty_S316_L008_R1_001.fastq.gz"
second_file = "empty_S316_L008_R2_001.fastq.gz"
single_file = "/home/dc/Downloads/single.fastq.gz"

startTime = time.time()
read_line = full_read
num_at = 0
num_plus = 0
function_lookup = {0:(process_header,read_line,num_at,adapters),1:(process_DNA,read_line),2:(process_plus,read_line,num_plus),3:(process_phred,read_line)}

with gzip.open(PATH+first_file,"rb") as fh1, gzip.open(PATH+second_file,"rb") as fh2,gzip.open(single_file, 'wb') as f_out:
    for x,y in zip(fh1,fh2):
        readline1 = x.strip()
        readline2 = x.strip()
        full_read = readline1 + readline2
        #print(full_read)
        #f_out.write(full_read)
        read_line = full_read.decode('UTF-8')
        func = function_lookup.get(num_read%4)
        if(num_at % 4 ==0):
            func[0](read_line,num_at,adapters)
        num_at += 1
    f_out.close()
    fh1.close()
    fh2.close()
print(time.time()-startTime)    